In [2]:
# Import packages
from pypdf import PdfReader
import gradio as gr
from agents import Agent, Runner, function_tool, OpenAIChatCompletionsModel, set_tracing_disabled
set_tracing_disabled(disabled=True)

In [3]:
import os
from openai import AsyncAzureOpenAI, OpenAIError
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(override=True)

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o-mini")

# Initialize Azure OpenAI client with Entra ID authentication
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(),
    "https://cognitiveservices.azure.com/.default"
)

openai_client = AsyncAzureOpenAI(
    azure_endpoint=endpoint,
    azure_ad_token_provider=token_provider,
    api_version="2025-01-01-preview",
)

In [4]:
reader = PdfReader("files/WS2_Architecture_proposal.pdf")
architecture_proposal = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        architecture_proposal += text

In [5]:
print(architecture_proposal)

Revision History
Version Revision date Status Summary of changes By
0.1 Concept Initial version Piethein Strengholt
0.2 26-05-2026 Concept Second version Discussed with Rob Visser, Tjerrie Smit, Sonja Miljoen, John van Iersel, Arno Goosen
0.2 26-05-2026 Concept Incorporating feedback from PWC and International stakeholders Informed: Bob Zondervan, Adrian Matei, Imre Sztanó, Frank Eijsink
0.3 12-06-2025 Concept Revised different scenarios, added extra requirements Review Jack Jonathans
0.4 16-06-2025 For information Discussed with McKinsey, Katalin Tatar
0.5 26-06-2025 Review Feeback WS2 working group Jack Jonathans, Marcel van Dijk, Arno Goosen, Sonja Miljoen, Hein Peters
0.6 26-06-2025 For information Informed: Wouter Wijnmalen, Maarten de Regt, Laurens van Beurden, Jorik Blaas-Sigmond
0.7 04-07-2025 Discussion Discussed with Tjeerd Bosklopper, Maurice Koopman, Guido Bosch, Rob Visser, Tjerrie Smit
0.8 16-07-2025 Discussion Changes for TPAB Discussed within TPAB
Document ManagementMis

In [6]:
adr_template = """---
title: ADR Template
---

---
MADR template version: 3.0.0 
MADR template source: https://github.com/adr/madr
---

# ADR Template

## Title. 
*mandatory*  
The title assigns a {name} to the AD so that it can be identified and searched for easily. Ideally, it should convey the essence of the problem solved and the solution chosen.

## Metadata.
*mandatory*  

The metadata elements are:

| Attribute Name | Description | Format |mandatory optional|
| -------------- | ----------- | ------ |------|
| status | status of the architecture decision | proposed \\| accepted \\| rejected \\| deprecated \\| superseded by | mandatory |
| date | when the decision was last updated | {YYYY-MM-DD} | mandatory |
| deciders | lists everyone involved in the decision |  | mandatory |
| consulted | lists everyone whose opinions are sought and with whom there is a two-way communication |  | optional |
| informed | lists everyone who is kept up-to-date on progress in one-way communication |  | optional |

### NN additions  
*mandatory*

| Attribute Name | Description | Format |mandatory optional|
| -------------- | ----------- | ------ |------|
| Identifier | Unique identifier of the decision | < integer > | mandatory |
| Organizational Scope | Identification of the organizational scope to which the architecture decision applies. This scope comprises of one or more predefined organizational units. | NN Group \\| < country > \\| < BU > \\| < country >\\\\< BU > \\| < country >\\\\< BU >\\\\< LoB > | mandatory |
| Functional Scope | Identification of the functional scope to which the architecture decision applies. This scope comprises of one or more domains, subdomains or digital products, preferably predefined, but free text is acceptable as a minimal viable solution. | < domain > \\| < domain >\\\\< subdomain > \\| < domain >\\\\< subdomain >\\\\< digital product > | mandatory |

## Context and Problem Statement.  
*mandatory*  
Describes the context and problem statement in a few sentences. One may want to articulate the problem in form of a question or provide an illustrative story that invites to a conversation. Links to collaboration boards or issue management systems can go here too.

## Decision Drivers.  
*optional*  
Desired qualities, forces, faced concerns are identified here:

- {decision driver n}

## Considered Options.  
*mandatory*  
This section lists the alternatives (or choices) investigated:

- {title/name of option n}

The template recommends listing the chosen option first (as a project-wide convention). One needs to make sure to list options that can solve the given problem in the given context (as documented in Section “Context and Problem Statement”). They should do so on the same level of abstraction; a mistake we have seen in practice is that a technology is compared with a product, or an architectural style with a protocol specification and its implementations. Pseudo-alternatives sometimes can be found too, but do not help.

## Chosen Option.  
*mandatory*  
Here, the chosen option is referred to by its title. A justification should be given as well: {name of option 1} because {justification}. Some examples of justifications are: it is the only option that meets a certain k.o. criterion/decision driver; it resolves a particular force; it comes out best when comparing options. 

## Consequences.  
*mandatory*  
This section discusses how problem and solution space look like after the decision is made (and enforced).

Positive and negative consequences are listed as “Good, because …” and “Bad, because …”, respectively. An example for a positive consequence is an improvement of a desired quality. A negative consequence might be extra effort or risk during implementation.

## Validation.  
*optional*  
This optional section describes how the implementation of/compliance with the ADR is evaluated (aka enforced), for instance by way of a review or a test. See “A Definition of Done for Architectural Decision Making” for related hints.

## Pros and Cons of the Options.  
*mandatory*  
Here, the alternatives that address the problem can be explained and analyzed more thoroughly.

The template advises to provide an example or a description of the option. Then, “Good” and “Bad” options properties are asked for. For noteworthy “Neutral” arguments, the template suggests the form

Neutral (w.r.t.), because {argument}.

## More Information.  
*optional*  
Here, one might want to provide additional evidence for the decision outcome (possibly including assumptions made) and/or document the team agreement on the decision (including the confidence level) and/or define how this decision should be realized and when it should be re-visited (the optional “Validation” section may also cover this aspect). Links to other decisions and resources might appear in this section as well.

Include a reference to the Design Doc that this ADR has originated from. While the ADR is lightweight, the design doc contains more detailed descriptions and diagrams.
"""


In [7]:
system_prompt = f"You are an experienced enterprise architect. You assist in writing Architecture Decision Records (ADRs) \
Given the following context, generate an Architecture Decision Record (ADR) in Markdown format. Include the problem, decision, status, alternatives considered, pros/cons, and consequences. \
You are provided via the chat with instructions about what to write the ADR for. \
You must adhere to the structure of the following template: {adr_template}"

system_prompt += f"\n\n## Architecture document:\n{architecture_proposal}\n\n"
system_prompt += f"With this context, please chat with the user."

system_prompt += f"Crucial Rules:\
- You must allow the user to make revisions on the ADR you write.\
- You must ask the user whether the ADR is ready to be handed off, and if so, you must hand it off to the store_adr tool."


In [8]:
system_prompt

"You are an experienced enterprise architect. You assist in writing Architecture Decision Records (ADRs) Given the following context, generate an Architecture Decision Record (ADR) in Markdown format. Include the problem, decision, status, alternatives considered, pros/cons, and consequences. You are provided via the chat with instructions about what to write the ADR for. You must adhere to the structure of the following template: ---\ntitle: ADR Template\n---\n\n---\nMADR template version: 3.0.0 \nMADR template source: https://github.com/adr/madr\n---\n\n# ADR Template\n\n## Title. \n*mandatory*  \nThe title assigns a {name} to the AD so that it can be identified and searched for easily. Ideally, it should convey the essence of the problem solved and the solution chosen.\n\n## Metadata.\n*mandatory*  \n\nThe metadata elements are:\n\n| Attribute Name | Description | Format |mandatory optional|\n| -------------- | ----------- | ------ |------|\n| status | status of the architecture dec

In [9]:
@function_tool
def store_adr(name, content):
    # Define the file name and content  
    file_name = name
    
    # Open the file in write mode ('w')  
    with open('files\\' + file_name + '.md', 'w') as file:  
        file.write(content)
    return {"recorded": "ok"}

In [10]:
# Let's look at it
store_adr

FunctionTool(name='store_adr', description='', params_json_schema={'properties': {'name': {'title': 'Name'}, 'content': {'title': 'Content'}}, 'required': ['name', 'content'], 'title': 'store_adr_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001B59A791940>, strict_json_schema=True, is_enabled=True)

In [11]:
tools = [store_adr]

In [12]:
tools

[FunctionTool(name='store_adr', description='', params_json_schema={'properties': {'name': {'title': 'Name'}, 'content': {'title': 'Content'}}, 'required': ['name', 'content'], 'title': 'store_adr_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001B59A791940>, strict_json_schema=True, is_enabled=True)]

In [13]:
async def chat(message, history):
    try:
        # Make an agent with name, instructions, model
        agent = Agent(
            name="ADR Writer", 
            instructions=system_prompt,
            model=OpenAIChatCompletionsModel(
                model="gpt-4o-mini",
                openai_client=openai_client
            ),
            tools=tools
        )
        result = await Runner.run(agent, message)
        print(result)
        return result.final_output
    except OpenAIError as e:
        print(f"OpenAI API Error: {str(e)}")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")

In [14]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
